In [1]:
# download example data
# !pip install gdown
from downloaddata import download_example_data
download_example_data()


Downloading...
From: https://drive.google.com/uc?id=1noRtbC5fxBlnO7YnvktjIDhFI61PdOSB
To: /tmp/TCGA-21-5784-01Z-00-DX1.tif
100%|██████████| 363M/363M [00:11<00:00, 30.3MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1jkTp0IJHHpmLd1yDO1L3KRFJgm0STh0d
To: /tmp/TCGA-21-5784-01Z-00-DX1.xml
100%|██████████| 9.67k/9.67k [00:00<00:00, 12.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nLdKzSLq79mon1RCevgEmG59E8Oq9JhZ
To: /tmp/TCGA-21-5784-01Z-00-DX1_tb_mask.tif
100%|██████████| 318k/318k [00:00<00:00, 5.35MB/s]


In [3]:
from wholeslidedata.iterators import create_batch_iterator
from wholeslidedata.image.wholeslideimagewriter import WholeSlideMaskWriter
from wholeslidedata.image.wholeslideimage import WholeSlideImage
import time
from pprint import pprint
from tqdm.notebook import tqdm

In [4]:
mode='training'
training_iterator = create_batch_iterator(mode=mode,
                                          user_config='./configs/slidingwindowconfig.yml',
                                          presets=('slidingwindow',),
                                          cpus=4, 
                                          number_of_batches=-1, 
                                          return_info=True)

spacing =0.5
tile_size = 1024
wsm_writer = WholeSlideMaskWriter()
with WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1.tif') as wsi:
    shape = wsi.shapes[wsi.get_level_from_spacing(spacing)]

wsm_writer.write(path='./test2.tif', spacing=spacing, dimensions=shape, tile_shape=(tile_size,tile_size))
for x_batch, y_batch, info in tqdm(training_iterator):
    point = info['sample_references'][0]['point']
    c, r = point.x+tile_size//2, point.y+tile_size//2
    wsm_writer.write_tile(tile=y_batch[0],coordinates=(int(c),int(r)), mask=y_batch[0])
wsm_writer.save()
training_iterator.stop()

Creating: ./test2.tif....
Spacing: 0.5
Dimensions: (36851, 25353)
Tile_shape: (1024, 1024)
